<a href="https://colab.research.google.com/github/K-Domar/MyRepo/blob/main/Diamond_MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [3]:
diamond = pd.read_csv('https://raw.githubusercontent.com/K-Domar/MyRepo/main/diamonds.csv')